In [1]:
import json
import numpy as np

from scipy.stats import poisson
from scipy.optimize import minimize

import warnings
warnings.filterwarnings('ignore')

In [2]:
def generate_games(clubs, filename):
    games = list()
    for club1 in clubs:
        for club2 in clubs:
            if club1 == club2: continue
            games.append(f'{club1} vs. {club2}\n')

    with open(filename, 'w') as f:
        f.writelines(games)

def preprocessing(competition, year):
    with open(f'../data/BrazilianSoccerData/results/processed/{competition}_{year}_games.json', 'r') as f:
        data = json.load(f)

    inx = dict()
    played_games = dict()
    inx_count = 0
    for game in data:
        game = str(game).zfill(3)
        home, away, result = data[game]['Home'], data[game]['Away'], data[game]['Result']
        result = result.split(' X ')
        result = [int(x) for x in result]
        if home not in played_games: played_games[home] = list()
        played_games[home].append([away, result])
        if home not in inx:
            inx[home] = dict()
            inx[home]['Atk'] = inx_count
            inx_count += 1
            inx[home]['Def'] = inx_count
            inx_count += 1

    generate_games(list(played_games.keys()), f'{competition}_{year}.csv')
    return played_games, inx

def likelihood(parameters, played_games, inx):
    lik = 0
    for home in played_games:
        for game in played_games[home]:
            away, result = game
            mu0 = parameters[inx[home]['Atk']] / parameters[inx[away]['Def']]
            mu1 = parameters[inx[away]['Atk']] / parameters[inx[home]['Def']]
            lik -= poisson.logpmf(result[0], mu0)
            lik -= poisson.logpmf(result[1], mu1)

    return lik

def optimize_parameters(competition, year):
    played_games, inx = preprocessing(competition, year)
    parameters = np.random.random(40)
    bounds = [(0, None) for _ in parameters]
    bounds[0] = (1, 1)
    res = minimize(likelihood, parameters, args = (played_games, inx), bounds = bounds)
    parameters = res.x
    for club in inx:
        for force in inx[club]:
            inx[club][force] = parameters[inx[club][force]]

    parameters = inx
    with open(f'parameters_{competition}_{year}.json', 'w') as f: json.dump(parameters, f)

In [3]:
optimize_parameters('Serie_A', 2023)